# <b>U.S.-Medical-Insurance-Costs


## Import libraries

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

## Make a DataFrames

In [71]:
observations = pd.read_csv('observations.csv')
species_info = pd.read_csv('species_info.csv')

## <b>Exploratory Data Analysis (EDA)

### EDA for `observations`

### Data Inspection



In [72]:
observations.head()

,scientific_name,park_name,observations
0,Vicia benghalensis,Great Smoky Mountains National Park,68
1,Neovison vison,Great Smoky Mountains National Park,77
2,Prunus subcordata,Yosemite National Park,138
3,Abutilon theophrasti,Bryce National Park,84
4,Githopsis specularioides,Great Smoky Mountains National Park,85


In [73]:
print("shape: {shape} \n".format(shape = observations.shape))

observations.info()

shape: (23296, 3) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23296 entries, 0 to 23295
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   scientific_name  23296 non-null  object
 1   park_name        23296 non-null  object
 2   observations     23296 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 546.1+ KB


### Missing Values and Duplicates

In [74]:
missing_values = observations.isnull().sum()
duplicates = observations.duplicated().sum()

print("missing_values: \n{ms} \n\nduplicates: {d}".format(ms = missing_values,d = duplicates))


missing_values: 
scientific_name    0
park_name          0
observations       0
dtype: int64 

duplicates: 15


### Drop Duplicates

In [76]:
observations = observations.drop_duplicates()
duplicates = observations.duplicated().sum()

print("duplicates:", duplicates)

duplicates: 0
